Prior to running this notebook run: `ollama serve &`. This will start the Ollama server and allow you to interact with it through this notebook

In [1]:
import dspy
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from typing import List, Dict, Union

from utils import json_to_dataframe, json_to_string_list

/opt/miniconda3/envs/nlp-from-scratch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
filepath = '../../data/vector_veterinary_imaging_2.json'

df = json_to_dataframe(filepath) 
rad_strings = json_to_string_list(filepath)

In [3]:
df

,case_identifier,findings,conclusions_and_recommendations
0,181153,Orthogonal pelvis and orthogonal right shoulde...,1. Medial right mildly comminuted acetabular f...
1,181413,"Three view whole body images dated April 14, 2...",The material within the stomach and small inte...
2,181821,Three view thoracic radiographs (total of 5 th...,No aggressive osseous changes are noted. The b...
3,181886,Orthogonal images of the right pelvic limb are...,"1. Chronic right calcaneal tendonopathy, with ..."
4,181911,Lateral abdomen and pelvis images are provided...,"1. Numerous small urinary cystoliths, non-obst..."
...,...,...,...
1756,274208,Three view thorax and three view abdomen image...,Aggressive osseous change of the L6 vertebral ...
1757,274229,Orthogonal thorax and three view abdomen image...,Right cranial pulmonary mass. This is most lik...
1758,274244,"Liver: Diffusely homogenously hyperechoic, oth...",At least one gastric mural nodule extending in...
1759,274249,Ventrodorsal pelvis and orthogonal stifles ima...,"Right coxofemoral subluxation, progressive fro..."


In [4]:
findings = list(df['findings'])
conclusions = list(df['conclusions_and_recommendations'])

## Retrieval

In [5]:
class SentenceTransformerRetriever(dspy.Retrieve):
    def __init__(self, model: str, findings: List[str], conclusions: List[str], k: int):
        self.model = model if isinstance(model, SentenceTransformer) else SentenceTransformer(model, trust_remote_code=True)
        self.findings = findings
        self.conclusions = conclusions
        self.k = k
        self.embeddings = None
        self.init_embeddings()

    def init_embeddings(self):
        self.embeddings = self.model.encode(self.findings)

    def forward(self, query: str, k: int) -> List[Dict[str, Union[str, float]]]:
        query_embedding = self.model.encode([query])
        similarities = cosine_similarity(query_embedding, self.embeddings)[0]
        top_k_indices = np.argsort(similarities)[-k:][::-1]

        results = []
        for idx in top_k_indices:
            results.append({
                'finding': self.findings[idx],
                'conclusion': self.conclusions[idx],
                'score': float(similarities[idx])
            })

        return results

In [6]:
vectorizer = "sentence-transformers/all-MiniLM-L6-v2"
# vectorizer = "dunzhang/stella_en_400M_v5"

In [7]:
retriever_model = SentenceTransformerRetriever(model=vectorizer, findings=findings, conclusions=conclusions, k=10)

/opt/miniconda3/envs/nlp-from-scratch/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Batches: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 56/56 [00:08<00:00,  6.28it/s]


In [8]:
findings[1]

'Three view whole body images dated April 14, 2023 are provided for review. The lungs are well-inflated. The cardiopulmonary structures are normal. No pulmonary nodules or intrathoracic lymphadenopathy are identified. The cardiac silhouette and associated vasculature are normal. The pleural space and diaphragmatic margin are normal. The stomach contains a moderate amount of amorphous granular soft tissue opaque material. The stomach is normal in position. No gastric wall thickening or mass effects are appreciated.the small intestine contains a mild amount of similarly textured soft tissue opaque material mixed with gas and fluid. Two populations of small intestine are not identified. The colon contains a moderate amount of granular formed fecal material. The peritoneal and retroperitoneal serosal detail is adequate. The visible hepatic, splenic, and renal silhouette margins are normal. The urinary bladder is moderately filled. The osseous structures are within normal limits.'

In [9]:
retriever_model.forward(query=findings[1], k=2)

Batches: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  9.39it/s]


[{'finding': 'Three view whole body images dated April 14, 2023 are provided for review. The lungs are well-inflated. The cardiopulmonary structures are normal. No pulmonary nodules or intrathoracic lymphadenopathy are identified. The cardiac silhouette and associated vasculature are normal. The pleural space and diaphragmatic margin are normal. The stomach contains a moderate amount of amorphous granular soft tissue opaque material. The stomach is normal in position. No gastric wall thickening or mass effects are appreciated.the small intestine contains a mild amount of similarly textured soft tissue opaque material mixed with gas and fluid. Two populations of small intestine are not identified. The colon contains a moderate amount of granular formed fecal material. The peritoneal and retroperitoneal serosal detail is adequate. The visible hepatic, splenic, and renal silhouette margins are normal. The urinary bladder is moderately filled. The osseous structures are within normal limit

## Language model

In [10]:
language_model = "qwen2.5"
# language_model = "gemma2"

In [11]:
ollama_model = dspy.OllamaLocal(
    base_url='http://127.0.0.1:11434',
    timeout_s=500,
    model=language_model,
    model_type='text',
    max_tokens=1024,
    num_ctx=1024,
    temperature=0.7,
    top_p=0.8
)

In [12]:
# Configure DSPy to use Ollama
dspy.settings.configure(lm=ollama_model, rm=retriever_model)

In [13]:
class GenerateConclusions(dspy.Signature):
    """Given a radiology finding and similar examples, generate an appropriate conclusions and recommendations section.
    The response should maintain a professional medical tone and follow the style of the examples."""

    finding = dspy.InputField(desc="Findings section of the radiology report.")
    similar_examples = dspy.InputField(desc="Similar examples of findings and corresponding conclusions and recommendations sections.")
    conclusions = dspy.OutputField(desc="The conclusions and recommendations section. Give the findings section above.")

In [14]:
class RadiologyModule(dspy.Module):
    def __init__(self, retriever):
        super().__init__()
        self.generate_conclusion = dspy.Predict(GenerateConclusions)
        self.retriever = retriever

    def forward(self, finding: str) -> dict:
        # Retrieve similar examples
        retrieved = self.retriever(finding, k=3)
        
        # Format examples for prompt
        examples_text = ""
        for i, ex in enumerate(retrieved, 1):
            examples_text += f"Example {i}:\n"
            examples_text += f"Finding: {ex['finding']}\n"
            examples_text += f"Conclusion: {ex['conclusion']}\n\n"

        # Generate new conclusion
        prediction = self.generate_conclusion(
            finding=finding,
            similar_examples=examples_text
        )

        return {
            'finding': finding,
            'generated_conclusion': prediction.conclusions,
            'similar_examples': retrieved
        }

## Full pipeline

In [15]:
def setup_rad_pipeline(filepath: str, vectorizer: str = "sentence-transformers/all-MiniLM-L6-v2", k: int = 3):
    """
    Set up the complete radiology report generation pipeline
    """
    # Load data
    df = json_to_dataframe(filepath)
    findings = list(df['findings'])
    conclusions = list(df['conclusions_and_recommendations'])
    
    # Initialize retriever
    retriever = SentenceTransformerRetriever(
        model=vectorizer,
        findings=findings,
        conclusions=conclusions,
        k=k
    )
    
    # Create and return the radiology module
    return RadiologyModule(retriever)

## Example

In [16]:
filepath = '../../data/vector_veterinary_imaging_2.json'
rad_pipeline = setup_rad_pipeline(filepath)

/opt/miniconda3/envs/nlp-from-scratch/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Batches: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 56/56 [00:06<00:00,  9.23it/s]


In [17]:
# Example finding
test_finding = """
The thoracic cavity demonstrates normal cardiac silhouette size and shape. 
The pulmonary vasculature appears within normal limits. 
There is a mild interstitial pattern noted in the caudodorsal lung fields.
No evidence of pleural effusion is noted.
"""

In [18]:
result = rad_pipeline(test_finding)
    
print("Generated Conclusion:")
print(result['generated_conclusion'])
print("\nSimilar Examples Used:")
for i, example in enumerate(result['similar_examples'], 1):
    print(f"\nExample {i} (Similarity Score: {example['score']:.3f}):")
    print(f"Finding: {example['finding']}")
    print(f"Conclusion: {example['conclusion']}")

Batches: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  8.64it/s]


Generated Conclusion:
### Conclusions

The radiological findings demonstrate a normal cardiac silhouette size and shape with no evidence of significant chamber enlargement, cardiomegaly, or other structural abnormalities. The pulmonary vasculature appears within normal limits, and the lungs are well-inflated without any interstitial or alveolar patterns suggestive of disease processes such as pneumonitis, fibrosis, or infection.

A mild interstitial pattern is noted in the caudodorsal lung fields, which may represent non-specific changes such as reactive hyperinflation or minor inflammatory processes. However, given the absence of other concerning features like pleural effusion, nodules, or significant parenchymal abnormalities, this finding does not indicate a specific pathological process.

### Recommendations

1. **Follow-Up**: Given the normal overall appearance of both the heart and lungs, routine follow-up imaging may be warranted in 6-12 months to monitor for any subtle changes 

## Remove the example in question from retrieval

As we go through a handful of examples, we want to make sure we don't include the example itself in the retrieval set. But it is fine to include all other examples. 

In [ ]:
import random
from typing import List, Dict, Union, Optional
import pandas as pd

class SentenceTransformerRetrieverWithExclusion(dspy.Retrieve):
    def __init__(self, model: str, findings: List[str], conclusions: List[str], k: int):
        self.model = model if isinstance(model, SentenceTransformer) else SentenceTransformer(model, trust_remote_code=True)
        self.findings = findings
        self.conclusions = conclusions
        self.k = k
        self.embeddings = None
        self.excluded_indices = set()
        self.init_embeddings()

    def init_embeddings(self):
        self.embeddings = self.model.encode(self.findings)
        
    def set_excluded_indices(self, indices: Optional[List[int]] = None):
        """Set indices to exclude from retrieval"""
        self.excluded_indices = set(indices or [])
        
    def clear_excluded_indices(self):
        """Clear all excluded indices"""
        self.excluded_indices = set()

    def forward(self, query: str, k: int) -> List[Dict[str, Union[str, float]]]:
        query_embedding = self.model.encode([query])
        similarities = cosine_similarity(query_embedding, self.embeddings)[0]
        
        # Create mask for excluded indices
        mask = np.ones_like(similarities, dtype=bool)
        if self.excluded_indices:
            mask[list(self.excluded_indices)] = False
        
        # Get top k indices excluding masked indices
        masked_similarities = similarities.copy()
        masked_similarities[~mask] = -np.inf
        top_k_indices = np.argsort(masked_similarities)[-k:][::-1]

        results = []
        for idx in top_k_indices:
            results.append({
                'finding': self.findings[idx],
                'conclusion': self.conclusions[idx],
                'score': float(similarities[idx])
            })

        return results

def setup_rad_pipeline_with_exclusion(filepath: str, vectorizer: str = "sentence-transformers/all-MiniLM-L6-v2", k: int = 3):
    """
    Set up the radiology pipeline with exclusion capability
    """
    df = json_to_dataframe(filepath)
    findings = list(df['findings'])
    conclusions = list(df['conclusions_and_recommendations'])
    
    retriever = SentenceTransformerRetrieverWithExclusion(
        model=vectorizer,
        findings=findings,
        conclusions=conclusions,
        k=k
    )
    
    return RadiologyModule(retriever), df

def run_evaluation_with_exclusion(filepath: str, num_examples: int = 5, seed: Optional[int] = None):
    """
    Run inference on a specified number of examples, excluding each example from its own retrieval set
    
    Args:
        filepath: Path to the JSON data file
        num_examples: Number of examples to evaluate
        seed: Random seed for reproducibility
        
    Returns:
        DataFrame containing the evaluation results
    """
    if seed is not None:
        random.seed(seed)
        
    # Setup pipeline with exclusion capability
    rad_pipeline, df = setup_rad_pipeline_with_exclusion(filepath)
    
    # Randomly select examples
    total_examples = len(df)
    selected_indices = random.sample(range(total_examples), min(num_examples, total_examples))
    
    results = []
    
    for idx in selected_indices:
        # Get the example
        finding = df['findings'].iloc[idx]
        actual_conclusion = df['conclusions_and_recommendations'].iloc[idx]
        
        # Set the current example to be excluded from retrieval
        rad_pipeline.retriever.set_excluded_indices([idx])
        
        # Run inference
        result = rad_pipeline(finding)
        
        # Clear exclusion for next iteration
        rad_pipeline.retriever.clear_excluded_indices()
        
        # Store results
        results.append({
            'index': idx,
            'finding': finding,
            'actual_conclusion': actual_conclusion,
            'generated_conclusion': result['generated_conclusion'],
            'similar_examples': result['similar_examples']
        })
        
    return pd.DataFrame(results)

## TODO 

Pull out a handful of examples of prompts, actual conclusions, and predicted conclusions and have the model judge how close the actual and predicted responses are and also to come up with a new set of prompts that might work better.

In [ ]:
filepath = '../../data/vector_veterinary_imaging_2.json'

# Run evaluation on 5 random examples
results_df = run_evaluation_with_exclusion(filepath, num_examples=5, seed=42)

# Print results
for idx, row in results_df.iterrows():
    print(f"\nExample {idx + 1}:")
    print("Finding:")
    print(row['finding'])
    print("\nActual Conclusion:")
    print(row['actual_conclusion'])
    print("\nGenerated Conclusion:")
    print(row['generated_conclusion'])
    print("\nSimilar Examples Used:")
    for i, example in enumerate(row['similar_examples'], 1):
        print(f"\nReference {i} (Similarity Score: {example['score']:.3f}):")
        print(f"Finding: {example['finding']}")
        print(f"Conclusion: {example['conclusion']}")
    print("\n" + "="*80)

## TODO 

incorporate the code below

In [ ]:
import json
from typing import List, Dict, Union
import pandas as pd
import dspy
from dspy.teleprompt import BootstrapFewShot, ValueEstimator
import numpy as np

def save_evaluation_results(results_df: pd.DataFrame, output_path: str, format: str = 'json'):
    """
    Save evaluation results for LLM analysis
    
    Args:
        results_df: DataFrame containing evaluation results
        output_path: Path to save the results
        format: Either 'json' or 'csv'
    """
    # Prepare data for export
    export_data = []
    for _, row in results_df.iterrows():
        export_row = {
            'finding': row['finding'],
            'actual_conclusion': row['actual_conclusion'],
            'generated_conclusion': row['generated_conclusion'],
            'reference_examples': [
                {
                    'finding': ex['finding'],
                    'conclusion': ex['conclusion'],
                    'similarity_score': float(ex['score'])
                } for ex in row['similar_examples']
            ]
        }
        export_data.append(export_row)
    
    # Save in specified format
    if format.lower() == 'json':
        with open(output_path, 'w') as f:
            json.dump(export_data, f, indent=2)
    else:  # csv
        # Flatten the reference examples
        flat_data = []
        for item in export_data:
            flat_item = {
                'finding': item['finding'],
                'actual_conclusion': item['actual_conclusion'],
                'generated_conclusion': item['generated_conclusion']
            }
            for i, ref in enumerate(item['reference_examples'], 1):
                flat_item[f'ref_{i}_finding'] = ref['finding']
                flat_item[f'ref_{i}_conclusion'] = ref['conclusion']
                flat_item[f'ref_{i}_similarity'] = ref['similarity_score']
            flat_data.append(flat_item)
        pd.DataFrame(flat_data).to_csv(output_path, index=False)

class ConclusionScorer(dspy.Signature):
    """Rate the quality and accuracy of generated radiology conclusions compared to actual conclusions."""
    
    finding = dspy.InputField()
    generated_conclusion = dspy.InputField()
    actual_conclusion = dspy.InputField()
    
    score = dspy.OutputField(desc="Score from 0-1 indicating similarity of content and style")
    reasoning = dspy.OutputField(desc="Explanation of the score and suggestions for improvement")

class RadiologyValueEstimator(ValueEstimator):
    def __init__(self):
        super().__init__()
        self.scorer = dspy.Predict(ConclusionScorer)
    
    def forward(self, example, pred, trace=None):
        score = self.scorer(
            finding=example.finding,
            generated_conclusion=pred.conclusions,
            actual_conclusion=example.actual_conclusion
        )
        return float(score.score)

def optimize_rad_prompt(filepath: str, num_bootstrap_examples: int = 10):
    """
    Use DSPy's teleprompter to optimize the radiology prompt
    
    Args:
        filepath: Path to the dataset
        num_bootstrap_examples: Number of examples to use for bootstrapping
    """
    # Load data
    df = json_to_dataframe(filepath)
    
    # Prepare training data
    train_data = [
        dspy.Example(
            finding=row['findings'],
            actual_conclusion=row['conclusions_and_recommendations']
        ).with_inputs('finding')
        for _, row in df.iterrows()
    ]
    
    # Initialize teleprompter components
    bootstrapper = BootstrapFewShot(
        demo_retriever='bm25',  # or 'sbert' for semantic search
        k=num_bootstrap_examples
    )
    
    value_estimator = RadiologyValueEstimator()
    
    # Create compiler configuration
    config = dspy.TelepromptConfig(
        metric=value_estimator,
        max_bootstrapping_iterations=3,
        max_rounds=5
    )
    
    # Initialize and run teleprompter
    teleprompter = dspy.Teleprompter(
        GenerateConclusions,
        bootstrapper=bootstrapper,
        config=config
    )
    
    # Optimize the prompt
    optimized_program = teleprompter.compile(
        train_data=train_data,
        eval_data=train_data[:100]  # Use subset for evaluation
    )
    
    return optimized_program

# Example usage:
if __name__ == "__main__":
    filepath = '../../data/vector_veterinary_imaging_2.json'
    
    # Run evaluation and save results
    results_df = run_evaluation_with_exclusion(filepath, num_examples=20, seed=42)
    save_evaluation_results(results_df, 'evaluation_results.json')
    
    # Optimize prompt using teleprompter
    optimized_program = optimize_rad_prompt(filepath)
    
    # Print optimized prompt
    print("Optimized Prompt:")
    print(optimized_program.signature.instructions)